In [33]:
import numpy as np

Y = [[1, 1, 2],
     [-1, 1, 3],
     [-1, 1, 4],
     [1, 2, 3],
     [1, 2, 4],
     [1, 3, 4]]

In [34]:
def function(point):
    alpha = 5
    return np.log(1/(1+np.exp(-Y[0][0]*(alpha-np.linalg.norm(point[Y[0][1]] - point[Y[0][1]])))))

In [35]:
def grad(point):
    alpha = 5
    return (-2*Y[0][0]*(points[Y[0][1]][0]-points[Y[0][2]][0])*np.exp(-Y[0][0]*(alpha-np.linalg.norm(points[Y[0][1]],points[Y[0][2]]))))/(1+np.exp(-Y[0][0]*(alpha-np.linalg.norm(points[Y[0][1]],points[Y[0][2]]))))

In [36]:
function({1: np.array([-2, 0]),2: np.array([1, 2]),3: np.array([4, 3]),4: np.array([4, 3])
})

-0.006715348489117944

In [37]:
grad({1: np.array([-2, 0]),2: np.array([1, 2]),3: np.array([4, 3]),4: np.array([4, 3])
})

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [39]:
from scipy.optimize import check_grad
check_grad(function, grad, [1.5, -1.5])

ModuleNotFoundError: No module named 'scipy'